In [ ]:
# EDA 진행
- 데이터 확인 
- 일자별 
- 시간대별 
- 요일별 

In [1]:
# import plotly.plotly as py
import chart_studio.plotly as iplot
import cufflinks as cf
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
warnings.filterwarnings("ignore")

plt.style.use('ggplot')
print(cf.__version__)

%config InlineBackend.figure_format = 'retina'
cf.go_offline()

0.17.3


In [13]:
%%time
query = """
SELECT 
    DATETIME_TRUNC(pickup_datetime, hour) as pickup_hour,
    count(*) as cnt
FROM `bigquery-public-data.new_york_taxi_trips.tlc_yellow_trips_2016` 
WHERE EXTRACT(MONTH from pickup_datetime) = 1
GROUP BY pickup_hour
ORDER BY pickup_hour
"""

df = pd.read_gbq(query=query, dialect='standard', project_id='nyc-taxi-302111', auth_local_webserver=True)

# 2016년 1월의 전체 trip 수 가져오기 

Wall time: 2.21 s


In [14]:
df

,pickup_hour,cnt
0,2016-01-01 00:00:00,25270
1,2016-01-01 01:00:00,28511
2,2016-01-01 02:00:00,25490
3,2016-01-01 03:00:00,21277
4,2016-01-01 04:00:00,15419
...,...,...
739,2016-01-31 19:00:00,17659
740,2016-01-31 20:00:00,14886
741,2016-01-31 21:00:00,14190
742,2016-01-31 22:00:00,12911


In [9]:
# dt형변환 
df['pickup_hour'] = pd.to_datetime(df['pickup_hour'])
df.tail()

,pickup_hour,cnt
739,2016-01-31 19:00:00,17659
740,2016-01-31 20:00:00,14886
741,2016-01-31 21:00:00,14190
742,2016-01-31 22:00:00,12911
743,2016-01-31 23:00:00,9258


In [15]:
df = df.set_index('pickup_hour')
df.head()

,cnt
pickup_hour,
2016-01-01 00:00:00,25270
2016-01-01 01:00:00,28511
2016-01-01 02:00:00,25490
2016-01-01 03:00:00,21277
2016-01-01 04:00:00,15419


In [16]:
df.iplot(kind='scatter', xTitle = 'Datetimes',yTitle='Demand',title='NYC Taxi Demand(2016-01)')

# 1월 24일에 유독 낮음 
# 블리자드 눈보라의 영향 (https://en.wikipedia.org/wiki/January_2016_United_States_blizzard)

In [17]:
# 일별 여행수 

df['date'] = df.index.date
df.groupby(['date'])[['cnt']].sum().iplot()

In [18]:
# 요일별 시간대별 trip 수 

df.head()

,cnt,date
pickup_hour,,
2016-01-01 00:00:00,25270,2016-01-01
2016-01-01 01:00:00,28511,2016-01-01
2016-01-01 02:00:00,25490,2016-01-01
2016-01-01 03:00:00,21277,2016-01-01
2016-01-01 04:00:00,15419,2016-01-01


In [19]:
# 요일별 dttime을 사용하여 추가하기 
df['weekday'] = df.index.weekday
df['hour'] = df.index.hour
df['weeknum'] = df.index.week